In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re

import warnings

warnings.filterwarnings('ignore')

In [2]:
STOP_WORD_FILE = 'Files/vietnamese-stopwords.txt'

In [3]:
with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
    stop_words = file.read()

stop_words = stop_words.split('\n')

In [4]:
data = pd.read_csv('Files/Product.csv', index_col=0)

In [5]:
data.head()

,product_id,product_name,description,product_rating,group,price
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,Dung lượng pin 300 Thời gian pin - Thời gian ...,4.0,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,77000
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,Dung lượng pin 2000mah Thời gian pin Thời gia...,4.5,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,132000
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,Thương hiệu Logitech Độ phân giải quang học 1...,4.8,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,299000
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,Thương hiệu Acome Xuất xứ thương hiệu Indones...,4.7,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,149000
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,Thương hiệu Apple Xuất xứ thương hiệu Mỹ Kích...,4.8,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,5090000


In [6]:
data['product'] = data['product_name'].astype(str) + data['description'].astype(str)

In [7]:
data.head()

,product_id,product_name,description,product_rating,group,price,product
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,Dung lượng pin 300 Thời gian pin - Thời gian ...,4.0,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,77000,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,Dung lượng pin 2000mah Thời gian pin Thời gia...,4.5,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,132000,Tai nghe bluetooth không dây F9 True wireless ...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,Thương hiệu Logitech Độ phân giải quang học 1...,4.8,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,299000,Chuột Không Dây Logitech M331 Silent - Hàng Ch...
3,57440329,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...,Thương hiệu Acome Xuất xứ thương hiệu Indones...,4.7,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,149000,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...
4,38458616,Tai Nghe Bluetooth Apple AirPods Pro True Wire...,Thương hiệu Apple Xuất xứ thương hiệu Mỹ Kích...,4.8,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,5090000,Tai Nghe Bluetooth Apple AirPods Pro True Wire...


In [8]:
data["Index"] = [i for i in range(data.shape[0])]

In [9]:
# word_tokenize
data['product_wt'] = data['product'].apply(lambda x: word_tokenize(x, format = 'text'))

In [10]:
data.to_csv('Files/Product_data.csv')

In [11]:
# Define helper functions for lookup and similarity
def find_product_from_index(index) :
    return data[data['Index'] == index]["product_name"].values[0]

def find_index_from_name(name) :
    return data[data["product_name"] == name]["Index"].values[0]

def content_based_product(name, percentage = 0, n = 5):
    # Get the index of the given product name
    ind = find_index_from_name(name)

    # Get all the similarity values for the product, and convert it to a list
    sim_product = list(enumerate(sim[ind]))

    # Sort the product based on similarity values
    names, score = map(list, zip(*sorted(sim_product, key = lambda x : x[1], 
                                        reverse = True)[1:]))
    
    # Get the number of recommended products
    if percentage == 0:
      rec_num = n
    else:
      # Get the number of given percentage
      rec_num = int((percentage/100) * len(titles))

    # Find the indexes of the top n most similar titles
    top_n_indexes = names[:rec_num]

    # Get the dataframe rows based on the indexes
    top_n_products = data.loc[data.index[top_n_indexes]]

    # Add the similarity scores to the dataframe
    top_n_products['Similarity-Score'] = score[:rec_num]

    # Return the dataframe
    return top_n_products[['Similarity-Score', 'product_name']]

In [12]:
find_product_from_index(1)

'Tai nghe bluetooth không dây F9 True wireless Dock Sạc có Led Báo Pin Kép'

In [13]:
name = 'Tai nghe bluetooth không dây F9 True wireless Dock Sạc có Led Báo Pin Kép'

### Doc2Vec

In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [15]:
tagged_data = [TaggedDocument(words = word_tokenize(doc), tags = [i]) 
                for i, doc in enumerate(data['product'])]

In [16]:
# Create a Doc2Vec model, which vectorizes our texts using a neural network
model_d2v = Doc2Vec(dm = 0, vector_size = 300, alpha = 0.05, min_alpha = 0.01,
                    min_count = 1)

# Build a vocabulary of the unique words in the whole corpus
model_d2v.build_vocab(tagged_data)

In [17]:
# Train the model using 20 epochs
for epoch in range(20):
    model_d2v.train(tagged_data, total_examples = model_d2v.corpus_count, 
                    epochs = model_d2v.epochs)

In [18]:
# Create an array filled with zeros, with the size of number of documents
# and vector size.    
doc_embeddings = np.zeros((data.shape[0], 300))

# Store the embedding vectors that was created by the model
for i in range(len(doc_embeddings)):
    doc_embeddings[i] = model_d2v.docvecs[i]

In [19]:
# Perform Cosine Similarity on our embedded texts
sim = cosine_similarity(doc_embeddings)
dv_sim = sim

# Content Based Recommendation
dv_recommendations = content_based_product(name)
dv_recommendations

,Similarity-Score,product_name
66,0.897091,"Tai Nghe Bluetooth Không Dây 5.0, AMOI F9, Cảm..."
23,0.860249,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
119,0.845998,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
635,0.781890,Tai Nghe Bluetooth Amoi F9 kèm Củ Sạc 1A và Cá...
134,0.459745,Tai Nghe Bluetooth TWS F9 Tai Nghe Nhét Hai T...


### Cosine similarity with TF-IDF

In [20]:
# Initialize tf-idf for product_wt
tfidf = TfidfVectorizer(analyzer='word', min_df=0, stop_words=stop_words)
tfidf_content = tfidf.fit_transform(data['product_wt'])

# Perform Cosine Similarity evaluation
sim = cosine_similarity(tfidf_content, tfidf_content)
tfidf_sim = sim

# Content Based Recommendation
tfidf_recommendations = content_based_product(name)
tfidf_recommendations

,Similarity-Score,product_name
66,0.930648,"Tai Nghe Bluetooth Không Dây 5.0, AMOI F9, Cảm..."
23,0.892896,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
119,0.871793,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
635,0.844326,Tai Nghe Bluetooth Amoi F9 kèm Củ Sạc 1A và Cá...
719,0.670654,Tai nghe Blutooth 5.0 kiêm dock sạc dự phòng- ...


### Cosine similarity with Count Vectorizer

In [21]:
# Initialize count vectorizer for product_wt
cv = CountVectorizer(stop_words = stop_words, ngram_range = (1, 2), 
                     analyzer = 'word')
cv_content = cv.fit_transform(data['product_wt'])

# Perform Cosine Similarity evaluation
sim = cosine_similarity(cv_content)
cv_sim = sim

# Content Based Recommendation
cv_recommendations = content_based_product(name)
cv_recommendations

,Similarity-Score,product_name
66,0.947926,"Tai Nghe Bluetooth Không Dây 5.0, AMOI F9, Cảm..."
23,0.906872,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
119,0.890952,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...
635,0.862546,Tai Nghe Bluetooth Amoi F9 kèm Củ Sạc 1A và Cá...
719,0.691710,Tai nghe Blutooth 5.0 kiêm dock sạc dự phòng- ...


- Choose cosine similarity with Count Vectorizer because it has higher Similarity-Score 

### Conclude:
- Content-based filtering (Gensim and cosine similarity): choose cosine similarity with Count Vectorizer. 

### Save the model

In [22]:
# Split the cosine similarity matrix into 2 partitions
partitions = np.array_split(cv_sim, 2)

# Save each partition separately
for i, partition in enumerate(partitions):
    np.save(f'ContentBased/cosine_similarity_partition_{i}.npy', partition)